In [1]:
from module.prepare import *
from itertools import product
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

import gc
import os
import re
import math
import sys
from collections import Counter
import random
from itertools import islice
import time
import configparser
import json
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import bokeh

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy as sp
from scipy import stats

import sklearn
from joblib import dump, load
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.manifold import *
import sklearn.tree as Tr 

import lightgbm as lgb

from module.metrics import *

D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def getCurrentTime():
    return datetime.datetime.strftime(datetime.datetime.fromtimestamp(time.time()),format='%Y-%m-%d-%H-%M-%S')


def LGBTuning(Xtrain,Xtest,Ytrain,Ytest,new_params=None):
    
    clf = lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=20,
#                              class_weight
                            )
    
    default_params = {
    'learning_rate': [0.1], 
    'boosting_type':['gbdt'], 
    'n_estimators': [500],
    'num_iterations':[1000],
    'max_bin':[256]
    }
    
    if new_params is not None:
        default_params.update(new_params)
    
    arg_str = ''
    for k,v in default_params.items():
        if type(v[0])==str:
            arg_str += k+'='+"'"+v[0]+"',"
        else:
            arg_str += k+'='+str(v[0])+","
    eval(
        'clf.'+clf.set_params.__name__+"("
            +arg_str.rstrip(',')+
            ")"
        )

    print('DEBUG:: tuning params\n',clf.get_params())
    clf.fit(Xtrain,Ytrain)
    Ypred = clf.predict(Xtest)
    score_train = clf.score(Xtrain,Ytrain)
    print('train score %f'%score_train)
    
    return [Ypred,Ytest,score_train,clf]

In [3]:
cv = 5
generalize_ratio = 1.0/cv
test_ratio = 1.0/cv
tuning_mode = False

if tuning_mode:
    cv = 1

tuning 1

In [4]:
def TuningParametersStage1(fname=getCurrentTime()+'-stage1.csv'):
    res = []
    for DATAID in [0,1,2,3,4,5]:
        INFO('data id %d'%DATAID)
        for RNA_K in range(3,7):
            for PROTEIN_K in range(3,7):
                for TOP_RATIO in np.linspace(0.93,0.99,5):
                    start_time = time.time()
                    [data,T] = ReadData(DATAID,PROTEIN_K,RNA_K)
                    [X,Y] = ToMatrix(data,'dense')
                    [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
                    [X_train,X_test,Y_train,Y_test] = \
                                RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=TOP_RATIO)
                    r = LGBTuning(X_train,X_test,Y_train,Y_test)
                    r = {
                        'DATAID': DATAID,
                        'test_score':scoreFunction(r[0],r[1]),
                        'train_score':r[2],
                        'RNA_K':RNA_K,
                        'PROTEIN_K':PROTEIN_K,
                        'TOP_RATIO':TOP_RATIO
                    }
                    print('DEBUG:: result ',r)
                    res.append(r)
                    end_time = time.time()
                    print('DEBUG:: time elapsed ',(end_time-start_time)/60)
    df = pd.DataFrame(data=res,columns=['DATAID','test_score','train_score','RNA_K','PROTEIN_K','TOP_RATIO'])
    df.to_csv(os.path.join('./result',fname))
    
    return


In [ ]:
# TuningParametersStage1()

In [9]:
import json
import re


params_1 = {
    0:[(5,5,0.99)],
    1:[(3,3,0.99)],
    2:[(4,4,0.96)],
    3:[(3,3,0.975)],
    4:[(4,3,0.96)],
    5:[(4,6,0.945)]
}



stage 2

In [6]:
fname_result2 = getCurrentTime()+'-stage2.csv'

tune_grid = [
    [{
        'learning_rate': [0.1], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[2000], ### 2000
        'num_leaves': [150,185,215,250], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.8,0.95], ### 0.75
        'subsample_freq':[0.9,0.95,1], ### 1
#         'lambda_l1': [1e-3,0]
    }],
    [{
        'learning_rate': [0.01], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[2000], ### 2000
        'num_leaves': [150,185,215,250], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.8,0.95], ### 0.75
        'subsample_freq':[0.9,0.95,1], ### 1
#         'lambda_l1': [1e-3,0]
    }],
    [{
        'learning_rate': [0.1], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[1000], ### 2000
        'num_leaves': [30,50,80,100], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.8,0.95], ### 0.75
        'subsample_freq': [0.9,0.95,1], ### 1
#         'lambda_l1': [1e-3,0]
     }],
    [{
        'learning_rate': [0.05], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[1000], ### 2000
        'num_leaves': [150,185,215,250], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.8,0.95], ### 0.75
        'subsample_freq': [0.9,0.95,1], ### 1
#         'lambda_l1': [1e-3,0]
     }],
    [{
        'learning_rate': [0.01], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[2000], ### 2000
        'num_leaves': [30,50,80,100], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.8,0.95], ### 0.75
        'subsample_freq': [0.9,0.95,1], ### 1
#         'lambda_l1': [1e-3,0]
     }],
    [{
        'learning_rate': [0.1], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[1000], ### 2000
        'num_leaves': [30,50,80,100], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.8,0.95], ### 0.75
        'subsample_freq': [0.9,1], ### 1
#         'lambda_l1': [1e-3,0]
     }]
]

tune_grid = list(map(lambda x:list(ParameterGrid(x)),tune_grid))


In [7]:
tuning_cv2 = 1
tuning_generalize_ratio2 = 1.0/tuning_cv2 if tuning_cv2!=1 else 0.2

df_columns = ['dataid','protein_k','rna_k','top_ratio','training_score','tune_param','scores']
df_result2 = pd.DataFrame([],columns=df_columns)

for _dataid in [0,1,2,3,4,5]:
    INFO('dataid %d'%_dataid)
    for global_params in params_1[_dataid]:
        start_time = time.time()
        ### get conf of dataid
        protein_k = global_params[0]
        rna_k = global_params[1]
        top_ratio = global_params[2]
        ### read data
        [data,T] = ReadData(_dataid,protein_k,rna_k)
        [X,Y] = ToMatrix(data,'dense')
        ### split dataset
        [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,tuning_generalize_ratio2)
        ### dimensionality reduction
        [X_train,X_test,Y_train,Y_test] = \
                    RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=top_ratio)
        for _cv in range(tuning_cv2):
            INFO('tuning cv %d'%_cv)
            for sp in tune_grid[_dataid]:
                sp = dict(map(lambda x:(x,[sp[x]]),sp))
#                 print(sp)
                tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,sp)
                tune_score = scoreFunction(tune_results[0],tune_results[1])
                r = pd.Series({
                                'dataid':_dataid,
                                'protein_k':protein_k,
                                'rna_k':rna_k,
                                'top_ratio':top_ratio,
                                'training_score':tune_results[2],
                                'tune_param':json.dumps(sp),
                                'scores':json.dumps(tune_score),
                })
#                 print(r)
                df_result2 = df_result2.append(r,ignore_index=True)
    
        end_time = time.time()
        print('DEBUG:: time elapsed ',(end_time-start_time)/60)
df_result2.to_csv(fname_result2)
            

INFO::dataid 0
read data 5 5
=============RETRIEVE TRIAN DATA=================
# DEBUG: # DEBUG: **************new dl 0***************
# DEBUG: READ SEQ FROM FILE
# DEBUG: READ CLUSTER FROM FILE
ERROR:: regex  
ERROR:: regex  
# DEBUG: READ PAIR FROM FILE
read line error 
# DEBUG: GENERATE NEGATIVE PAIR
# DEBUG: negative pair number 10411
INFO::count of negative pairs10411
# DEBUG: PAIR UNION
# DEBUG: EXTRACT FEATURES--PROTEIN
# DEBUG: EXTRACT FEATURES--RNA
# DEBUG: K-MER CALCULATION
# DEBUG: FEATURE UNION
# DEBUG: GARBAGE COLLECTION
MATRIX TRANSFORMATION
DEBUG:: total features count  32068
data shape 20822 32068
rf raw data fit score 1.000000
INFO::dimension remained 31907 0.990000
dimension remained 31907
INFO::tuning cv 0
DEBUG:: tuning params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_job

D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 1.000000
DEBUG:: tuning params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 20, 'num_leaves': 200, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': 1, 'num_iterations': 1000, 'max_bin': 256}


D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 1.000000
DEBUG:: tuning params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 20, 'num_leaves': 500, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': 1, 'num_iterations': 1000, 'max_bin': 256}


...

train score 1.000000
DEBUG:: tuning params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 1000, 'n_jobs': 20, 'num_leaves': 200, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': 1, 'num_iterations': 2000, 'max_bin': 256}


D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 1.000000
DEBUG:: tuning params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 1000, 'n_jobs': 20, 'num_leaves': 500, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': 1, 'num_iterations': 2000, 'max_bin': 256}


D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 1.000000
DEBUG:: time elapsed  1668.04904037714


分析

In [13]:
acc = [json.loads(row['scores'])['acc'] for idx,row in df_result2.iterrows()]
auc = [json.loads(row['scores'])['auc'] for idx,row in df_result2.iterrows()]

In [16]:
df_result2['acc'] = acc
df_result2['auc'] = auc

In [36]:
a = df_result2.groupby(by=['dataid']).agg({'acc':max}).reset_index()
df_result2

,dataid,protein_k,rna_k,top_ratio,training_score,tune_param,scores,acc,auc
0,0,5,5,0.990,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9481392557022809, ""auc"": 0.949098347...",0.948139,0.949098
1,0,5,5,0.990,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9481392557022809, ""auc"": 0.948981476...",0.948139,0.948981
2,0,5,5,0.990,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9459783913565426, ""auc"": 0.946874711...",0.945978,0.946875
3,0,5,5,0.990,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9478991596638655, ""auc"": 0.948960938...",0.947899,0.948961
4,0,5,5,0.990,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.948859543817527, ""auc"": 0.9496840423...",0.948860,0.949684
...,...,...,...,...,...,...,...,...,...
319,5,4,6,0.945,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9408670520231214, ""auc"": 0.941069465...",0.940867,0.941069
320,5,4,6,0.945,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9406358381502891, ""auc"": 0.940838135...",0.940636,0.940838
321,5,4,6,0.945,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9404046242774566, ""auc"": 0.940553966...",0.940405,0.940554
322,5,4,6,0.945,1.0,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.9404046242774566, ""auc"": 0.940606805...",0.940405,0.940607


In [45]:
b = a.join(df_result2.set_index(['dataid','acc']),on=['dataid','acc'],how='inner',lsuffix='_left', rsuffix='_right')
b.to_csv('./result/stage2-join.csv')
b

,dataid,acc,protein_k,rna_k,top_ratio,training_score,tune_param,scores,auc
0,0,0.948860,5,5,0.990,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.948859543817527, ""auc"": 0.9496840423...",0.949684
0,0,0.948860,5,5,0.990,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.948859543817527, ""auc"": 0.9496840423...",0.949684
1,1,0.898230,3,3,0.990,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8982300884955752, ""auc"": 0.898789511...",0.898790
1,1,0.898230,3,3,0.990,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8982300884955752, ""auc"": 0.898789511...",0.898790
1,1,0.898230,3,3,0.990,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8982300884955752, ""auc"": 0.898789511...",0.898790
1,1,0.898230,3,3,0.990,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8982300884955752, ""auc"": 0.898789511...",0.898790
2,2,0.887755,4,4,0.960,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8877551020408163, ""auc"": 0.889730639...",0.889731
2,2,0.887755,4,4,0.960,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8877551020408163, ""auc"": 0.889730639...",0.889731
2,2,0.887755,4,4,0.960,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8877551020408163, ""auc"": 0.889730639...",0.889731
2,2,0.887755,4,4,0.960,1.000000,"{""boosting_type"": [""gbdt""], ""learning_rate"": [...","{""acc"": 0.8877551020408163, ""auc"": 0.889730639...",0.889731


In [21]:
fname_optimal_stage1 = './result/2020-05-04-20-55-26-stage1.csv'
df = pd.read_csv(fname_optimal_stage1)
acc = [ float(re.findall('[0-9]+\.[0-9]+',x)[0]) for x in df['test_score'] ]
auc = [ float(re.findall('[0-9]+\.[0-9]+',x)[1]) for x in df['test_score'] ]
df['acc'] = acc
df['auc'] = auc

new_df1 = df.groupby(by=['DATAID','PROTEIN_K','RNA_K']).agg({'acc':np.max,'auc':np.max})
new_df2 = df.groupby(by=['DATAID','PROTEIN_K','RNA_K']).agg({'acc':np.mean,'auc':np.mean})

In [22]:
new_df1

acc       auc
DATAID PROTEIN_K RNA_K                    
0      3         3      0.951741  0.952266
                 4      0.954622  0.954944
                 5      0.951020  0.951664
                 6      0.950060  0.950587
       4         3      0.953181  0.954216
...                          ...       ...
5      5         6      0.939075  0.939077
       6         3      0.941040  0.941068
                 4      0.942775  0.942947
                 5      0.942254  0.942425
                 6      0.940867  0.940881

[96 rows x 2 columns]

In [ ]:
import xgboost as xgb
import catboost as cb

DATAID = 0
PROTEIN_K = params_1[DATAID][0][0]
RNA_K = params_1[DATAID][0][1]
topRatio = params_1[DATAID][0][2]
[data,T] = ReadData(DATAID,PROTEIN_K,RNA_K)
[X,Y] = ToMatrix(data,'dense')
[X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
[X_train,X_test,Y_train,Y_test] = \
    RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=0.96)
r1 = LGBTuning(X_train,X_test,Y_train,Y_test)

read data 5 5
=============RETRIEVE TRIAN DATA=================
# DEBUG: # DEBUG: **************new dl 0***************
# DEBUG: READ SEQ FROM FILE
# DEBUG: READ CLUSTER FROM FILE
ERROR:: regex  
ERROR:: regex  
# DEBUG: READ PAIR FROM FILE
read line error 
# DEBUG: GENERATE NEGATIVE PAIR
# DEBUG: negative pair number 10411
INFO::count of negative pairs10411
# DEBUG: PAIR UNION
# DEBUG: EXTRACT FEATURES--PROTEIN
# DEBUG: EXTRACT FEATURES--RNA
# DEBUG: K-MER CALCULATION


In [ ]:
scoreFunction(r1[0],r1[1])

In [ ]:
estimators = [
    ('cb', Tr.DecisionTreeClassifier()),
    ('lgb', lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=12,
                            )),
    ('xgb',xgb.XGBClassifier()),
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegressionCV()
)

clf.fit(X_train,Y_train)

In [ ]:
Y_pred = clf.predict(X_test)
scoreFunction(Y_pred,Y_test)